In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

triplet loss function https://pytorch.org/docs/stable/nn.html#tripletmarginloss

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import pandas as pd

In [4]:
import  datatset
from torch.utils.data import DataLoader

In [5]:
# DataLoader

In [6]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]
df_train["augment"] = np.zeros(len(df_train),dtype=bool)

/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
df_train.head()

,file_name,category_id,augment
0,images/156/72/354106.jpg,15672,False
1,images/156/72/545181.jpg,15672,False
2,images/156/72/449419.jpg,15672,False
3,images/156/72/200223.jpg,15672,False
4,images/156/72/5327.jpg,15672,False


In [8]:
label_counts  = df["category_id"].value_counts()

In [9]:
num_calsses = len(label_counts) + 1

In [10]:
labels_ordered = label_counts.keys().tolist()

In [11]:
num_calsses

32094

In [12]:
ds = datatset.HerbDataSet(df_train, TRAIN_PATH,256,label="category_id", device="gpu")

In [13]:
data_loader = DataLoader(ds, batch_size=8, shuffle=False)

# Tracking

In [14]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/"logs/run-2")

In [15]:
import metrics
from train import Training

In [16]:
acc =  metrics.Accuracy()
f1 = metrics.F1()
manager = metrics.MetricManager([acc, f1], writer)

In [17]:
import torch.nn as nn

In [18]:
import triplet_loss
import focal_loss

In [34]:
import focal_loss

In [19]:
get_trip_loss = triplet_loss.HardTripletLoss(hardest=True, squared=True)

In [20]:
get_focal_loss = focal_loss.FocalLoss()
# get_focal_loss2 = focal_loss.FocalLoss2()

# Train

In [21]:
class Trainer(Training):
    def __init(self,  metrics, loss, optim, data, epochs, model, save_dir):
        super(self).__init__(self,  metrics, loss, optim, data, epochs, model, save_dir)
        
    def train_step(self, batch):
        data, labels = batch
        embeddings, logits = self.model(data)
        
        trip_loss = get_trip_loss(embeddings, labels)
        fl_loss = get_focal_loss(logits, labels)
        loss = trip_loss + fl_loss
        
#         if self.metrics.writer:
#             self.metrics.writer.add_scalar("loss", loss, self.step)
#             self.metrics.writer.add_scalar("trip_loss", trip_loss, self.step)
#             self.metrics.writer.add_scalar("cros_entropy_loss", get_focal_loss, self.step)

#         print(f'trip_loss: {trip_loss}')
#         print(f'fl_loss: {fl_loss}')
        print(f'trip_loss: {trip_loss}')
#         print(f'loss: {loss}')

        self.optim.zero_grad()  # zero gradients
        loss.backward()  # calculate gradients
        self.optim.step()  # updated weights


In [22]:
import torch
import torchvision.models as models
import model

In [23]:
m = model.get_triplet_model(models.resnet50(pretrained=False), p1=0, p2=0) # no drop out, trying to over fit

In [24]:
model_path = "/home/shaun/personal/kaggle-data/nybg2020/models-1/triplet-81000.pth"

In [25]:
m.load_state_dict(torch.load(model_path))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [26]:
m.cuda()

TripletsModel(
  (base): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Co

In [27]:
layer_lrs = [
    {'params': m.base.layer4.parameters(), 'lr': 1e-4},
    {'params': m.embeddings.parameters(), 'lr': 1e-3},
    {'params': m.classifier.parameters(), 'lr': 1e-3}
]

In [28]:
optimizer = torch.optim.Adam(layer_lrs)

In [29]:
model_path=TRAIN_PATH/"models/"

In [30]:
! export CUDA_LAUNCH_BLOCKING=1

In [31]:
Trainer(manager, None, optimizer, data_loader, 3, m, model_path).run()

Epoch 0/3


tensor([15672, 15672, 15672, 15672, 15672, 15672, 15672, 15672],
       device='cuda:0')
trip_loss: 0.10000000149011612


/home/shaun/personal/kaggle/herbarium_2020/focal_loss.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(logits)


tensor([15672, 15672, 15672, 15672, 15672, 15672, 15672, 15672],
       device='cuda:0')
trip_loss: 0.10000000149011612
tensor([15672, 15672, 11524, 11524, 11524, 11524, 11524, 11524],
       device='cuda:0')
trip_loss: 106306.984375
tensor([11524, 11524, 11524, 11524, 11524, 11524, 11524, 11524],
       device='cuda:0')
trip_loss: 0.10000000149011612
tensor([11524, 11524, 11524, 11524, 11524, 11524, 11524, 11524],
       device='cuda:0')
trip_loss: 0.10000000149011612
tensor([11524, 11524, 11524, 11524, 11524, 11524, 11524, 11524],
       device='cuda:0')
trip_loss: 0.10000000149011612
tensor([11467, 11467, 11467, 11467, 15660, 15660, 15660, 15660],
       device='cuda:0')
trip_loss: 242864.59375


In [32]:
ce_loss2: 0.05258801952004433
ce_loss: 0.05258801952004433

In [33]:
106307/ 100000, 242865/ 100000, 404139 / 100000


(1.06307, 2.42865, 4.04139)